In [1]:
#importing Libraries 
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier



In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

##reading the data
dat = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [3]:
X, y = dat.iloc[:, 1:-1] , dat.iloc[:,-1]

In [4]:
scalar = StandardScaler()
X_nw = scalar.fit_transform(X)

In [5]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6, 7]
        }

In [6]:
xg = xgb.XGBClassifier(objective ='binary:logistic' , colsample_bytree = 1.0 , learning_rate = 0.05,
                max_depth = 5, alpha = 10, n_estimators = 700 )

In [ ]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV( xg , param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs= 4, cv=skf.split(X_nw,y), verbose=3, random_state=1001 )
random_search.fit(X_nw, y)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] subsample=0.6, min_child_weight=1, max_depth=4, gamma=5, colsample_bytree=0.6 
[CV] subsample=0.6, min_child_weight=1, max_depth=4, gamma=5, colsample_bytree=0.6 
[CV] subsample=0.6, min_child_weight=1, max_depth=4, gamma=5, colsample_bytree=0.6 
[CV] subsample=0.6, min_child_weight=1, max_depth=4, gamma=5, colsample_bytree=0.6 


In [ ]:
from sklearn.metrics import accuracy_score
y_p = random_search.predict(X_nw)
accuracy_score(y, y_p)

In [ ]:
X_t, idd = test.iloc[:, 1:] , test.iloc[:,0]
Xt = scalar.fit_transform(X_t)

In [ ]:
y_t = random_search.predict(Xt)

In [ ]:
fn = pd.DataFrame()
fn['ID'] = idd
fn['class'] = y_t
fn.to_csv('sol_3.csv' , index= False)